In [1]:
from PIL import Image
from keras.applications import MobileNet, ResNet50
from keras import layers
from keras import models
from keras import backend as K
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification
from keras.utils import np_utils
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
import zipfile

Using TensorFlow backend.


### Data loading

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'Data/Tr/'
test_data_dir = 'Data/Te/'
img_height = 224
img_width = 224
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=45,
    width_shift_range=0.2,
    brightness_range=[0.3,1.5],
    horizontal_flip = True,
    vertical_flip = True) 


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
    test_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False) # set as validation data



Found 43486 images belonging to 2 classes.
Found 10752 images belonging to 2 classes.


### Neural Net construction

Decided to use ResNet50 with pretrained weights from imagenet dataset.

In [3]:
conv_base = ResNet50(weights='imagenet', include_top = False, 
                     input_shape = (224,224,3))

/jet/var/python/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [4]:
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Also tried MobileNet, but decided to use the first one.

In [17]:
#base_model = MobileNet(weights='imagenet',include_top=False)

/jet/var/python/lib/python3.6/site-packages/keras_applications/mobilenet.py:206: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


17227776/17225924 [==============================] - 1s 0us/step


In [21]:
#x=base_model.output
#x=layers.GlobalAveragePooling2D()(x)
#x=layers.Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
#x=layers.Dense(1024,activation='relu')(x) #dense layer 2
#x=layers.Dense(512,activation='relu')(x) #dense layer 3
#preds=layers.Dense(1,activation='sigmoid')(x) #final layer with softmax activation


In [23]:
#model2=models.Model(inputs=base_model.input,outputs=preds)

In [24]:
#for layer in base_model.layers:
    #layer.trainable = False

Created AUC metric in order to use in the training, because it is not among the standard ones.

In [5]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [6]:
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

Just feeling safer with checkpoints.

In [7]:
filepath="mynetfin-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_auc', verbose=1, save_best_only=True, mode='max', save_weights_only=False,period = 5)

Here is the model with our own last layer. It is also reasonable to freeze the layers (that what we wanted from the beginning) in order to have pretrained weights and save a lot of time.

In [8]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [9]:
for layer in model.layers[:1]:
    layer.trainable = False

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',auc])

In [11]:
nb_epochs = 100
model.fit_generator(
    train_generator,
    steps_per_epoch = 200,
    validation_data = validation_generator, 
    validation_steps = 100,
    epochs = nb_epochs, 
    verbose = 1,
    callbacks = [checkpoint])

Epoch 1/100
200/200 [==============================] - 127s 637ms/step - loss: 0.5437 - acc: 0.7195 - auc: 0.7207 - val_loss: 3.0485 - val_acc: 0.0000e+00 - val_auc: 0.5608
Epoch 2/100
200/200 [==============================] - 130s 651ms/step - loss: 0.4734 - acc: 0.7719 - auc: 0.5045 - val_loss: 1.9806 - val_acc: 0.3747 - val_auc: 0.5172
Epoch 3/100
200/200 [==============================] - 120s 598ms/step - loss: 0.4500 - acc: 0.7894 - auc: 0.5715 - val_loss: 0.0233 - val_acc: 1.0000 - val_auc: 0.6502
Epoch 4/100
200/200 [==============================] - 124s 621ms/step - loss: 0.4367 - acc: 0.7973 - auc: 0.7003 - val_loss: 1.7991 - val_acc: 0.3600 - val_auc: 0.7001
Epoch 5/100
200/200 [==============================] - 140s 698ms/step - loss: 0.4184 - acc: 0.8058 - auc: 0.6876 - val_loss: 2.3260 - val_acc: 0.0147 - val_auc: 0.6808

Epoch 00005: val_auc improved from -inf to 0.68079, saving model to mynetfin-05-0.01.hdf5
Epoch 6/100
200/200 [==============================] - 120s 

Epoch 45/100
200/200 [==============================] - 127s 633ms/step - loss: 0.3099 - acc: 0.8613 - auc: 0.7588 - val_loss: 3.4524 - val_acc: 0.0000e+00 - val_auc: 0.7562

Epoch 00045: val_auc improved from 0.75220 to 0.75616, saving model to mynetfin-45-0.00.hdf5
Epoch 46/100
200/200 [==============================] - 121s 606ms/step - loss: 0.3004 - acc: 0.8620 - auc: 0.7537 - val_loss: 1.0221 - val_acc: 0.6947 - val_auc: 0.7537
Epoch 47/100
200/200 [==============================] - 117s 584ms/step - loss: 0.3083 - acc: 0.8658 - auc: 0.7555 - val_loss: 0.1601 - val_acc: 1.0000 - val_auc: 0.7562
Epoch 48/100
200/200 [==============================] - 121s 603ms/step - loss: 0.2998 - acc: 0.8676 - auc: 0.7570 - val_loss: 2.9359 - val_acc: 0.0400 - val_auc: 0.7557
Epoch 49/100
200/200 [==============================] - 128s 639ms/step - loss: 0.2974 - acc: 0.8672 - auc: 0.7542 - val_loss: 2.2970 - val_acc: 0.3347 - val_auc: 0.7525
Epoch 50/100
200/200 [==============================

200/200 [==============================] - 118s 589ms/step - loss: 0.2653 - acc: 0.8844 - auc: 0.7842 - val_loss: 1.9720 - val_acc: 0.3600 - val_auc: 0.7844
Epoch 89/100
200/200 [==============================] - 134s 669ms/step - loss: 0.2646 - acc: 0.8844 - auc: 0.7838 - val_loss: 2.1275 - val_acc: 0.0147 - val_auc: 0.7838
Epoch 90/100
200/200 [==============================] - 117s 586ms/step - loss: 0.2743 - acc: 0.8739 - auc: 0.7839 - val_loss: 0.2249 - val_acc: 1.0000 - val_auc: 0.7841

Epoch 00090: val_auc improved from 0.78027 to 0.78414, saving model to mynetfin-90-1.00.hdf5
Epoch 91/100
200/200 [==============================] - 117s 584ms/step - loss: 0.2672 - acc: 0.8864 - auc: 0.7844 - val_loss: 0.6071 - val_acc: 0.7200 - val_auc: 0.7846
Epoch 92/100
200/200 [==============================] - 126s 631ms/step - loss: 0.2682 - acc: 0.8862 - auc: 0.7850 - val_loss: 1.6276 - val_acc: 0.0000e+00 - val_auc: 0.7855
Epoch 93/100
200/200 [==============================] - 122s 610m

In [12]:
model.save('finalnet.hdf5')

After training we see a decent result, our net demonstrated up to 88% accuracy on training set with area under curve 0.79, while accuracy on the validation is 93%.

It was tough